In [ ]:
ratingsFile = open('pruned_data.csv',encoding = 'utf8',errors='ignore')
users = {}
business = {}
user_counter = 0
business_counter = 0
row = []
col = []
data = []
for line in ratingsFile:
    lineValues = line.split(',')
    user_id = lineValues[0]
    business_id = lineValues[1]
    rating = float(lineValues[2])
    data.append(rating)
    if user_id not in users:
        row.append(user_counter)
        users[user_id] = user_counter
        user_counter += 1
    else:
        row_number = users[user_id]
        row.append(row_number)
    if business_id not in business:
        col.append(business_counter)
        business[business_id] = business_counter
        business_counter += 1
    else:
        col_number = business[business_id]
        col.append(col_number)
ratingsFile.close()

In [ ]:
import scipy.sparse as sp_sparse
import numpy as np
row = np.array(row)
col = np.array(col)
print(col)
print(row)
data = np.array(data)
M = sp_sparse.csr_matrix((data,(row,col)),shape=(len(users),len(business)))
M = M.T
print(M)
print(M.toarray())

In [ ]:
from scipy.sparse import csr_matrix
import random
print(M.get_shape)
non_zero_cells = csr_matrix.nonzero(M)
# print(non_zero_cells[0])
removed_cells = []
num_non_zero = len(non_zero_cells[0])
cells_to_remove = int(0.05*len(non_zero_cells[0]))
while cells_to_remove > 0:
    k = random.randint(0,num_non_zero-1)
    i = non_zero_cells[0][k]
    j = non_zero_cells[1][k]
    if (i,j) not in removed_cells:
        M[i,j] = 0
        removed_cells.append((non_zero_cells[0][i],non_zero_cells[1][i]))
        cells_to_remove -= 1
print(cells_to_remove)
print(non_zero_cells)
print(removed_cells)

In [ ]:
import sklearn.metrics as metrics
M_cosine_similarity = metrics.pairwise.cosine_similarity(M)
#M_cosine_similarity = csr_matrix(M_cosine_similarity)
print(len(business))
print(M_cosine_similarity.shape)

In [ ]:
import sys
import time
import copy
import numpy as np
from scipy.sparse import csc_matrix
M_csc = csr_matrix.tocsc(M)
np.fill_diagonal(M_cosine_similarity,0)
predictions = []
cell_counter = 0
start_time = time.time()
for removed_cell in removed_cells:

    topk = []
    k=20
    i = removed_cell[0]
    j = removed_cell[1]

    non_zero_ratings = csc_matrix.nonzero(M_csc[:,j])
    number = len(non_zero_ratings[0])

    mask_data = np.ones((number))
    mask_row = np.zeros((number),dtype=int)
    mask_col = non_zero_ratings[0]
    mask = sp_sparse.csr_matrix((mask_data,(mask_row,mask_col)),shape=(1,len(M_cosine_similarity[i,:])))

    similarity_row = M_cosine_similarity[i,:]
    similarity_row = sp_sparse.csr_matrix(similarity_row)

    similarity_row = similarity_row.toarray()
    mask = mask.toarray()
    result = similarity_row * mask

    new_mask = csr_matrix((1,len(M_cosine_similarity[i,:])), dtype=float)
    copied = copy.deepcopy(result)

    while k > 0 : #and len(np.nonzero(result))>k 
        index = np.argmax(result)
        # print(index)

        topk.append((index,result[0][index]))

        new_mask[0,index] = copied.item(0,index)

        result[0][index] = 0
        k -= 1

    sum_of_sim = 0
    sum_of_rating = 0

    sum_of_sim = np.sum(new_mask)

    sum_of_rating = new_mask * M[:,j]
    sum_of_rating = np.sum(sum_of_rating)
    cell_counter += 1

    # for element in topk:
    #     sum_of_sim += element[1]
    #     #print(M[element[0],j])
    #     sum_of_rating += M[element[0],j] * element[1]
    # if sum_of_rating == sumsim:
    #     cell_counter += 1
    prediction = sum_of_rating/sum_of_sim
    predictions.append((i,j,prediction,sum_of_rating))
    M[i,j] = prediction
    
end_time = time.time()
total_time = end_time -start_time
print(total_time)
print(cell_counter)

In [ ]:
print(len(predictions))
print(len(removed_cells))

In [ ]:
print(predictions)